# Text to image - OCR (optical character recognition) experiments

## Main Idea

    The goal of this project is to experiment with different ways to implement image-to-text. The motivation is personal, in order to speed up data entry for a team schedule updater. 
    
    The system from where the original text comes from is isolated from other computers, so it's necessary to photograph and re-type the original text in a Trello notebook. Currently, a commercial app is being used, downloaded from the playstore. theapp, however, is full of ads, and the speed of it is lacking. 
    
    There are 2 ways I will attempt to implement here:
    - Pytesseract: a ready-to-use package that I will import
    - My own model: based on what I learn from using pytesseract, I will attempt to develop my own model. 

### Pytesseract

Tesseract is a package built for OCR, and can very easily downloaded through pip. It is probably the easiest way to implement OCR in a project, and has support for many languages. Each language package can be download from their github repository, and installed individually. The library comes with english as a default

- Tesseract documentation: https://github.com/tesseract-ocr/tessdoc
- Tesseract language compatibility by version and code for each language: https://github.com/tesseract-ocr/tessdoc/blob/main/Data-Files-in-different-versions.md
- Code to download languages other than english:
    - Specific Language (3 letters):
            apt-get install tesseract-ocr-YOUR_LANG_CODE
    - All languages:
            apt-get install tesseract-ocr-all
    

In [4]:
# For Google Colab
!pip install pytesseract
!pip install pdf2image
!pip install pdftocairo


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pdftocairo (from versions: none)
ERROR: No matching distribution found for pdftocairo


In [ ]:
!pip install popper
!pip install tesseract

In [ ]:
!apt update
!apt-get install libnss3 libnss3-dev
!apt-get install libcairo2-dev libjpeg-dev libgif-dev
!apt-get install cmake libblkid-dev e2fslibs-dev libboost-all-dev libaudit-dev

!wget https://poppler.freedesktop.org/poppler-21.09.0.tar.xz;
!tar -xvf poppler-21.09.0.tar.xz;

!mkdir -p poppler-21.09.0/build && \

In [53]:
!  apt-get install tesseract-ocr-all

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr tesseract-ocr-afr tesseract-ocr-amh tesseract-ocr-ara
  tesseract-ocr-asm tesseract-ocr-aze tesseract-ocr-aze-cyrl tesseract-ocr-bel
  tesseract-ocr-ben tesseract-ocr-bod tesseract-ocr-bos tesseract-ocr-bre
  tesseract-ocr-bul tesseract-ocr-cat tesseract-ocr-ceb tesseract-ocr-ces
  tesseract-ocr-chi-sim tesseract-ocr-chi-sim-vert tesseract-ocr-chi-tra
  tesseract-ocr-chi-tra-vert tesseract-ocr-chr tesseract-ocr-cos
  tesseract-ocr-cym tesseract-ocr-dan tesseract-ocr-deu tesseract-ocr-div
  tesseract-ocr-dzo tesseract-ocr-ell tesseract-ocr-eng tesseract-ocr-enm
  tesseract-ocr-epo tesseract-ocr-est tesseract-ocr-eus tesseract-ocr-fao
  tesseract-ocr-fas tesseract-ocr-fil tesseract-ocr-fin tesseract-ocr-fra
  tesseract-ocr-frk tesseract-ocr-frm tesseract-ocr-fry tesseract-ocr-gla
  tesseract-ocr-gle tesseract-ocr-glg tesserac

In [ ]:
!mkdir -p poppler-21.09.0/build && \
cd poppler-21.09.0 && \
cmake  -DCMAKE_BUILD_TYPE=Release   \
       -DCMAKE_INSTALL_PREFIX=/usr  \
       -DTESTDATADIR=$PWD/testfiles \
       -DENABLE_UNSTABLE_API_ABI_HEADERS=ON && \
make && \
make install


In [15]:
from PIL import Image
import pytesseract

In [6]:


# Format home/vls/code/vsattamini/tst.jpeg from home to the end
dire=input("Type full path to image you want to convert to text\n (format is with forward slashes):")
im = Image.open(dire)

language=input("Type the language of the scanned document (3 letter ID):")
text = pytesseract.image_to_string(im, lang=language)
filename=input("Type output file name:)")
file1 = open(filename+"-"+language+".txt","w")
file1.write(text)
file1.close()
print("Done!") 

Type full path to image you want to convert to text
 (format is with forward slashes):x


FileNotFoundError: ignored

### Extra: Full document processing

Another opportunity to use this function presented itself as I was working on the original project. The following section was made as an addendum to the one above. I just needed to be able to send a PDF I found online to my kindle. The PDF wasn't in text, but in scanned images of the original book

I first used a 3rd party app to transform my PDF into images, and then looped the above model on the images. I then exported it in plain text and plan to convert it into EPUB to read.

In [16]:
# Necessary imports
from glob import glob
from pdf2image import convert_from_path
import os



In [17]:
pop_path = os.path.abspath(poppler.__file__)

NameError: ignored

In [34]:
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

In [42]:
!pip install tesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562571 sha256=da14add5e8cb67f811d1b1c5b4ecf4779556767a290312e7f9b25e2f84cf4bbf
  Stored in directory: /root/.cache/pip/wheels/6c/c5/81/8310cc52076953e53412ed1875a5e224c92940235bdcee21a2
Successfully built tesseract


In [44]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from glob import glob

import os

# os.chdir allows you to change directories, like cd in the Terminal
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [45]:
CURRENT_DIRECTORY = os.getcwd()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [54]:
# import module
def pdf_to_images(file):
    os.mkdir(f'{file}')
    pages = convert_from_path(f"{CURRENT_DIRECTORY}/{file}.pdf")
    for i in range(len(pages)):
        pages[i].save(f"{file}/{file} - Page {i+1}.jpg", 'JPEG')

In [55]:
def get_paths(file_name):
    files = glob(f'{file_name}/*.jpg')
    files.sort()
    return files

In [56]:
def extract_and_join_jpg(files, final_file_name, langua="eng"):
    final_text = []
    for i, path in enumerate(files):
        dire= CURRENT_DIRECTORY+"/"+path
        im = Image.open(dire)
        text = pytesseract.image_to_string(im, lang=langua)

        final_text.append(text)
        if i % 10 == 0:
            print(f"Image {i} done,{len(files) - i+1} images to go...")
    final_text = " ".join(final_text)
    
    filename= final_file_name
    file1 = open(filename+"-"+langua+".txt","w")
    file1.write(final_text)
    file1.close()
    return print("Done! Check your file")

In [59]:
!pip install pandoc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 7.0 MB/s eta 0:00:00
  Created wheel for pandoc: filename=pandoc-2.3-py3-none-any.whl size=33284 sha256=a378a38fffe78c3d7cfe6265bd76c5f64038fd5e1d7625bfeeec221db34a8153
  Stored in directory: /root/.cache/pip/wheels/69/e6/a1/1daa96d919c9e09a71473649b717b8da286f3f8d7719d1cfc5
Successfully built pandoc


In [ ]:
def txt_to_epub(txt):


In [57]:
def full_extraction_process(file_name, langua='eng'):
    pdf_to_images(file_name)
    files = get_paths(file_name)
    extract_and_join_jpg(files,file_name)
    return print('Fuly done')

In [58]:
# Testing on new book
full_extraction_process("Foucault_Michel_Madness_and_Civilization_A_History_of_Insanity_in_the_Age_of_Reason")

Image 0 done,318 images to go...
Image 10 done,308 images to go...
Image 20 done,298 images to go...
Image 30 done,288 images to go...
Image 40 done,278 images to go...
Image 50 done,268 images to go...
Image 60 done,258 images to go...
Image 70 done,248 images to go...
Image 80 done,238 images to go...
Image 90 done,228 images to go...
Image 100 done,218 images to go...
Image 110 done,208 images to go...
Image 120 done,198 images to go...
Image 130 done,188 images to go...
Image 140 done,178 images to go...
Image 150 done,168 images to go...
Image 160 done,158 images to go...
Image 170 done,148 images to go...
Image 180 done,138 images to go...
Image 190 done,128 images to go...
Image 200 done,118 images to go...
Image 210 done,108 images to go...
Image 220 done,98 images to go...
Image 230 done,88 images to go...
Image 240 done,78 images to go...
Image 250 done,68 images to go...
Image 260 done,58 images to go...
Image 270 done,48 images to go...
Image 280 done,38 images to go...
Ima

In [ ]:
os.mkdir('convert_from_path')

In [ ]:
import os

In [ ]:
"Foucault_Michel_Madness_and_Civilization_A_History_of_Insanity_in_the_Age_of_Reason"

## Model Creation

In the following section, I will attempt to create a model from scratch, training and tuning a model based on papers found in PapersWithCode and Arxiv

In [ ]:
# Importing Dataset from Hugging Face
from datasets import load_dataset


dataset = load_dataset("naver-clova-ix/synthdog-en")